# Julia is fast

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - C (hand-written)
    - C (hand-written with -ffast-math)
    - python (built-in)
    - python (numpy)
    - python (hand-written)
    - Julia (built-in)
    - Julia (hand-written)
    - Julia (hand-written with SIMD)
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [1]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Vector{Float64}:
 0.8236172835882443
 0.7930556960084199
 0.8121687114002252
 0.3202148302470471
 0.40489181388265505
 0.3189452290468108
 0.35744412567031203
 0.787539719254093
 0.5099890538580851
 0.21855473445387907
 0.9765711418134846
 0.6788863164966298
 0.35274932587189634
 ⋮
 0.027280901994840456
 0.3986174880157548
 0.5015448240323601
 0.25244889824216576
 0.3156487407482682
 0.11787613103043482
 0.5947374606128472
 0.5437635583092146
 0.8240759054105569
 0.8449372152191272
 0.09906903174713333
 0.5495738101832662

In [2]:
sum(a)

5.000741204935156e6

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

In [3]:
@time sum(a)

  0.008562 seconds (1 allocation: 16 bytes)


5.000741204935156e6

The `@time` macro can yield noisy results, so it's not our best choice for benchmarking!

Luckily, Julia has a `BenchmarkTools.jl` package to make benchmarking easy and accurate:

In [4]:
# using Pkg
# Pkg.add("BenchmarkTools")

In [5]:
using BenchmarkTools  

#  1. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [6]:
# native C code inside julia
using Libdl
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [7]:
c_sum(a)

5.000741204935832e6

In [8]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

true

In [9]:
c_sum(a) - sum(a)  

6.761401891708374e-7

In [10]:
≈  # alias for the `isapprox` function

isapprox (generic function with 9 methods)

In [ ]:
?isapprox

We can now benchmark the C code directly from Julia:

In [12]:
c_bench = @benchmark c_sum($a)

BenchmarkTools.Trial: 449 samples with 1 evaluation.
 Range (min … max):  10.391 ms …  14.979 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.064 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   11.134 ms ± 404.593 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

            ▄▃█▅▆▂▃▃▄ ▁                                         
  ▂▂▃▂▃▆▆▄▇████████████▅▅▅▆▄▄▃▄▃▃▃▁▃▁▂▂▃▂▁▁▁▁▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▂ ▃
  10.4 ms         Histogram: frequency by time         13.1 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [13]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

C: Fastest time was 10.390732 msec


In [14]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

Dict{Any, Any} with 1 entry:
  "C" => 10.3907

# 2. C with -ffast-math

If we allow C to re-arrange the floating point operations, then it'll vectorize with SIMD (single instruction, multiple data) instructions.

In [17]:
const Clib_fastmath = tempname()   # make a temporary file

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = ccall(("c_sum", Clib_fastmath), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum_fastmath (generic function with 1 method)

In [18]:
c_fastmath_bench = @benchmark $c_sum_fastmath($a)

BenchmarkTools.Trial: 590 samples with 1 evaluation.
 Range (min … max):  7.568 ms …  12.490 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.526 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.462 ms ± 359.607 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                ▁   ▂    ▄▄█▂ ▂ ▂              
  ▃▂▃▁▁▂▄▄▃▃▄▄▅▄▄▄▃▃▄▄▅▅▄▅▃▄▄▄▇▅███▇██▇▇▇████▇███▇▆▆▄▆▄▄▃▄▃▃▅ ▄
  7.57 ms         Histogram: frequency by time        9.02 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [19]:
d["C -ffast-math"] = minimum(c_fastmath_bench.times) / 1e6  # in milliseconds

7.567622

In [20]:
d

Dict{Any, Any} with 2 entries:
  "C"             => 10.3907
  "C -ffast-math" => 7.56762

# 3. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [21]:
# using Pkg; Pkg.add("PyCall")
using PyCall

In [22]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [23]:
pysum(a)

5.000741204935832e6

In [24]:
pysum(a) ≈ sum(a)

true

In [25]:
py_list_bench = @benchmark $pysum($a)

BenchmarkTools.Trial: 6 samples with 1 evaluation.
 Range (min … max):  806.192 ms … 866.170 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     852.028 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   844.508 ms ±  22.100 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                        █                   █ █     █      █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁█▁▁▁▁▁█▁▁▁▁▁▁█ ▁
  806 ms           Histogram: frequency by time          866 ms <

 Memory estimate: 240 bytes, allocs estimate: 6.

In [26]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any, Any} with 3 entries:
  "C"               => 10.3907
  "Python built-in" => 806.192
  "C -ffast-math"   => 7.56762

# 4. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [ ]:
# using Pkg; Pkg.add("Conda")
# using Conda

In [28]:
# Conda.add("numpy")

In [ ]:
numpy_sum = pyimport("numpy")["sum"]

In [ ]:
py_numpy_bench = @benchmark $numpy_sum($a)

In [30]:
numpy_sum(a)

5.000741204935155e6

In [31]:
numpy_sum(a) ≈ sum(a)

true

In [32]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any, Any} with 4 entries:
  "C"               => 10.3907
  "Python numpy"    => 7.72976
  "Python built-in" => 806.192
  "C -ffast-math"   => 7.56762

# 5. Python, hand-written 

In [33]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x7fa6905a47c0>

In [34]:
py_hand = @benchmark $sum_py($a)

BenchmarkTools.Trial: 5 samples with 1 evaluation.
 Range (min … max):  1.048 s …   1.102 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.076 s              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.073 s ± 20.782 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █            █                █     █                   █  
  █▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.05 s         Histogram: frequency by time         1.1 s <

 Memory estimate: 240 bytes, allocs estimate: 6.

In [35]:
sum_py(a)

5.000741204935832e6

In [36]:
sum_py(a) ≈ sum(a)

true

In [37]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any, Any} with 5 entries:
  "C"                   => 10.3907
  "Python numpy"        => 7.72976
  "Python hand-written" => 1047.57
  "Python built-in"     => 806.192
  "C -ffast-math"       => 7.56762

# 6. Julia (built-in) 

## Written directly in Julia, not in C!

In [38]:
@which sum(a)

sum(a::AbstractArray; dims, kw...)
     @ Base reducedim.jl:994

In [39]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 581 samples with 1 evaluation.
 Range (min … max):  7.784 ms …  11.627 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.584 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.587 ms ± 263.218 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                  ▄█▆▅▅▅▄▄▃▄▃▁▂▃▁  ▂           
  ▇▅▅▇▅▁▅▅▅▇▆▄▁▆▅▄▇▆▆▇▇▅▆▆▁▅▆▅▄▆▆▁████████████████████▅▇▁▆▄▁▅ ▇
  7.78 ms      Histogram: log(frequency) by time       9.1 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [40]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any, Any} with 6 entries:
  "C"                   => 10.3907
  "Python numpy"        => 7.72976
  "Python hand-written" => 1047.57
  "Python built-in"     => 806.192
  "Julia built-in"      => 7.78446
  "C -ffast-math"       => 7.56762

# 7. Julia (hand-written) 

In [41]:
function mysum(A)   
    s = 0.0 # s = zero(eltype(a))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [42]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 432 samples with 1 evaluation.
 Range (min … max):  10.620 ms …  15.592 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.482 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   11.558 ms ± 467.356 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                ▂  █▃▄▅▂▄▄▂▁▂▃  ▂                               
  ▃▁▁▁▃▄▃▄▆▅▆▇▇██████████████████▇▇▄█▅▃▇▅▅▄▅▅▄▃▃▁▅▃▁▃▁▁▁▃▄▁▁▃▄ ▄
  10.6 ms         Histogram: frequency by time         12.9 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [43]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any, Any} with 7 entries:
  "C"                   => 10.3907
  "Python numpy"        => 7.72976
  "Julia hand-written"  => 10.6196
  "Python hand-written" => 1047.57
  "Python built-in"     => 806.192
  "Julia built-in"      => 7.78446
  "C -ffast-math"       => 7.56762

# 8. Julia (hand-written w. simd) 

In [44]:
function mysum_simd(A)   
    s = 0.0 # s = zero(eltype(A))
    @simd for a in A
        s += a
    end
    s
end

mysum_simd (generic function with 1 method)

In [45]:
j_bench_hand_simd = @benchmark mysum_simd($a)

BenchmarkTools.Trial: 580 samples with 1 evaluation.
 Range (min … max):  7.731 ms …   9.452 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.601 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.602 ms ± 213.031 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                  ▆██▇▇▆▄▄▅▃▂▃▃▁▁              
  ▆▁▁▁▆▁▆▆▄▄▆█▇▄▇▆▄▆▆▆▄▆▆▆▇▆▄█▄▆▄▄███████████████▆▇▆▇▄▄▆▁▁▄▁▄ █
  7.73 ms      Histogram: log(frequency) by time      9.18 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [46]:
mysum_simd(a)

5.000741204935168e6

In [47]:
d["Julia hand-written simd"] = minimum(j_bench_hand_simd.times) / 1e6
d

Dict{Any, Any} with 8 entries:
  "Julia hand-written simd" => 7.73099
  "C"                       => 10.3907
  "Python numpy"            => 7.72976
  "Julia hand-written"      => 10.6196
  "Python hand-written"     => 1047.57
  "Python built-in"         => 806.192
  "Julia built-in"          => 7.78446
  "C -ffast-math"           => 7.56762

# Summary

In [48]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

C -ffast-math...............7.6
Python numpy................7.7
Julia hand-written simd.....7.7
Julia built-in..............7.8
C..........................10.4
Julia hand-written.........10.6
Python built-in...........806.2
Python hand-written......1047.6
